## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sklearn

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from pathlib import Path
from IPython.display import Image, display, Video, HTML
from ipywidgets import interact, widgets

from signlens.params import *
from signlens.preprocessing import data, preprocess
from utils import plot_landmarks, model_utils

# reload automatically python functions outside notebook
%load_ext autoreload
%autoreload 2

2024-03-22 08:20:43.270902: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 08:20:44.591734: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Fetch data

In [2]:
# variables loaded from .env
print(f"DATA_FRAC : {DATA_FRAC}")
print(f"NUM_CLASSES : {NUM_CLASSES}")
print(f"MAX_SEQ_LEN : {MAX_SEQ_LEN}")

DATA_FRAC : 0.5
NUM_CLASSES : 10
MAX_SEQ_LEN : 100


In [3]:
NUM_CLASSES = 10

In [4]:
sequences_data = data.load_data_subset_csv(noface=True, balanced=True, n_classes=NUM_CLASSES)

✅ File with frames already exists, loaded matching 'sequence_id' rows.
✅ Filtered on n_frames = 100. Size reduced from 94477 to 86168 (91.2%)
✅ Filtered on n_classes = 10. Size reduced from 86168 to 3491 (4.1%)
✅ Balanced data, with average of 174.5 elements per class. Size reduced from 3491 to 1745 (50.0%)
✅ Loaded 3491 rows (1.8% of the original 94477 rows) from the dataset.


In [5]:
sequences_data.sign.value_counts()

sign
chair     175
book      175
before    175
go        175
drink     175
yes       174
who       174
fine      174
no        174
all       174
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

X_files = sequences_data.file_path
y = preprocess.label_dictionnary(sequences_data)

# Train test split
X_train_files, X_test_files, y_train, y_test = train_test_split(X_files, y, test_size=0.2, stratify=y)

# Train split into train and val
X_train_files, X_val_files, y_train, y_val = train_test_split(X_train_files, y_train, test_size=0.2, stratify=y_train)

In [7]:
X_train = preprocess.group_pad_sequences(X_train_files)
X_val = preprocess.group_pad_sequences(X_val_files)
X_test = preprocess.group_pad_sequences(X_test_files)


In [10]:
X_train.shape

(1116, 100, 75, 3)

In [11]:
y_train.shape

(1116, 10)

## Model

In [21]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers, regularizers

from tensorflow.keras.utils import to_categorical

In [32]:
normalizer = layers.Normalization(input_shape=(MAX_SEQ_LEN, N_LANDMARKS_NO_FACE, 3))
normalizer.adapt(X_train)


/home/bfrisque/.pyenv/versions/3.10.6/envs/signlens/lib/python3.10/site-packages/keras/src/layers/preprocessing/normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [60]:
# 1- RNN Architecture
model = Sequential()
model.add(layers.Reshape((MAX_SEQ_LEN, N_LANDMARKS_NO_FACE*3), input_shape=(MAX_SEQ_LEN, N_LANDMARKS_NO_FACE, 3)))
model.add(layers.Masking(mask_value=0.0))

model.add(layers.SimpleRNN(units=128, return_sequences=True))
model.add(layers.Dropout(0.3))
model.add(layers.LSTM(units=64))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))



model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_23 (Reshape)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_23 (Masking)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_17 (SimpleRNN)       │ (None, 100, 128)       │        45,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,370 (372.54 KB)

 Trainable params: 95,370 (372.54 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
# Fit model

es = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=32, 
                    verbose=1, 
                    callbacks = [es] # This will call the Early Stopping Criterion for each epoch
                   )


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 114ms/step - accuracy: 0.0959 - loss: 2.3754 - val_accuracy: 0.0929 - val_loss: 2.3138
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.1439 - loss: 2.3066 - val_accuracy: 0.1321 - val_loss: 2.3042
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.1718 - loss: 2.2513 - val_accuracy: 0.1286 - val_loss: 2.3157
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 117ms/step - accuracy: 0.1610 - loss: 2.2379 - val_accuracy: 0.1393 - val_loss: 2.2675
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - accuracy: 0.2012 - loss: 2.2111 - val_accuracy: 0.1500 - val_loss: 2.2593
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 117ms/step - accuracy: 0.1540 - loss: 2.2001 - val_accuracy: 0.1393 - val_loss: 2.2772
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 117ms/step - accuracy: 0.1833 - loss: 2.1506 - val_accuracy: 0.1786 - val_loss: 2.2003
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 117ms/step - accuracy: 0.1925 - loss: 2.1432 - val_accu

In [45]:
model_utils.plot_history_interactive(history)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Y Min:', max=1.0), FloatSli…

<function utils.model_utils.plot_history_interactive.<locals>.plot_hist(y_min, y_max, epoch_min, epoch_max)>

In [44]:
model_utils.plot_history_interactive(history)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Y Min:', max=1.0), FloatSli…

<function utils.model_utils.plot_history_interactive.<locals>.plot_hist(y_min, y_max, epoch_min, epoch_max)>

In [49]:
from signlens.model.model import *

In [70]:
model2 = initialize_model(num_classes=NUM_CLASSES)

model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model2.summary()


/home/bfrisque/.pyenv/versions/3.10.6/envs/signlens/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_27 (Reshape)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_27 (Masking)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_21 (SimpleRNN)       │ (None, 100, 128)       │        45,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,370 (372.54 KB)

 Trainable params: 95,370 (372.54 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
batch_size = 32
es = EarlyStopping(patience=10, restore_best_weights=True)


modelCheckpoint = ModelCheckpoint(
    MODEL_DIR + os.path.sep + "model_epoch_{epoch:02d}.keras",
    monitor="val_accuracy",
    verbose=0,
    save_freq=10*int(X_train.shape[0]/batch_size)
    )

LRreducer = ReduceLROnPlateau(monitor="val_accuracy", factor = 0.1, patience=5, verbose=1, min_lr=1e-6)
    

history2 = model2.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=batch_size, 
                    verbose=1, 
                    callbacks = [es,modelCheckpoint,LRreducer] # This will call the Early Stopping Criterion for each epoch
                   )


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - accuracy: 0.1037 - loss: 2.4163 - val_accuracy: 0.0821 - val_loss: 2.3183 - learning_rate: 0.0010
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - accuracy: 0.1109 - loss: 2.3279 - val_accuracy: 0.1179 - val_loss: 2.2966 - learning_rate: 0.0010
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 124ms/step - accuracy: 0.0934 - loss: 2.3313 - val_accuracy: 0.1286 - val_loss: 2.2937 - learning_rate: 0.0010
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.1076 - loss: 2.3169 - val_accuracy: 0.1464 - val_loss: 2.2742 - learning_rate: 0.0010
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.1517 - loss: 2.2697 - val_accuracy: 0.1036 - val_loss: 2.2763 - learning_rate: 0.0010
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - accuracy: 0.1512 - loss: 2.2658 - val_accuracy: 0.1607 - val_loss: 2.2468 - learning_rate: 0.0010
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - accuracy: 0.1541 - loss: 2.2

## Model import

In [15]:
from signlens.model.model import compile_model, initialize_model, train_model

In [18]:
model = initialize_model()
model = compile_model(model)
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_3 (Reshape)             │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_3 (Masking)             │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 100, 128)       │        45,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,370 (372.54 KB)

 Trainable params: 95,370 (372.54 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model, history = train_model(model,
                             X_train, y_train, 
                             epochs=100, 
                             validation_data=(X_val, y_val))


Training model...
Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 296ms/step - accuracy: 0.1918 - loss: 2.1523 - val_accuracy: 0.2143 - val_loss: 2.1513 - learning_rate: 1.0000e-06
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step - accuracy: 0.2071 - loss: 2.1694 - val_accuracy: 0.2143 - val_loss: 2.1513 - learning_rate: 1.0000e-06
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 279ms/step - accuracy: 0.2193 - loss: 2.1541 - val_accuracy: 0.2143 - val_loss: 2.1513 - learning_rate: 1.0000e-06
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 0.2080 - loss: 2.1448 - val_accuracy: 0.2143 - val_loss: 2.1512 - learning_rate: 1.0000e-06
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step - accuracy: 0.1916 - loss: 2.1769 - val_accuracy: 0.2143 - val_loss: 2.1512 - learning_rate: 1.0000e-06
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 306ms/step - accuracy: 0.2253 - loss: 2.1703 - val_accuracy: 0.2179 - val_loss: 2.1511 - learning_rate: 1.0000e-06
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 356ms/step 


KeyboardInterrupt

